In [4]:
import os
from openai import OpenAI

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv('Key_AzureOpenAI')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('Endpoint_AzureOpenAI')

os.environ['OPENAI_API_KEY'] = os.getenv('Key_OpenAI')

In [6]:
client = OpenAI()

In [38]:
filepath_AAPL = "../10k/APL_10k_2023Q4.pdf"
filepath_report1 = "../../US_Equity_Research/Boeing/Barclays Airbus Boeing Delivery tracker 8 Aug 2022.pdf"

file = client.files.create(
    file=open(filepath_AAPL, 'rb'),
    purpose='assistants'
)

print(file)

FileObject(id='file-0tAhpiBLCPw2v5CwIia7lk6Y', bytes=714094, created_at=1730655845, filename='APL_10k_2023Q4.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [39]:
assistant = client.beta.assistants.create(
    name = "PDF Helper",
    instructions = "You are my assistant who can answer questions from the given pdf",
    tools = [{"type":"file_search"}],
    model = "gpt-4o",
)

In [40]:
thread = client.beta.threads.create()
print(thread)

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "what do you know about this document?"
)

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= assistant.id

)

import time 

while True:
# Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    time.sleep(10)
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        break
    else:
        ### sleep again
        time.sleep(2)

for message in reversed(messages.data):
  print(message.role + ":" + message.content[0].text.value)

Thread(id='thread_K08CQLTVJSkVrA8RXnu3W5j6', created_at=1730655847, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
user:what do you know about this document?
assistant:I need you to upload the document you are referring to so I can analyze and provide information about it.
